# Project One

### Author：

In [1]:
!pip install wbdata

  Using cached wbdata-1.1.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached dateparser-1.2.2-py3-none-any.whl.metadata (29 kB)
  Using cached shelved_cache-0.3.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
Using cached wbdata-1.1.0-py3-none-any.whl (18 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
Using cached dateparser-1.2.2-py3-none-any.whl (315 kB)
Using cached shelved_cache-0.3.1-py3-none-any.whl (7.8 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Using cached tzlocal-5.3.1-py3-none-any.whl (18 kB)
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cach

## 1A Population Statistics

In [2]:
import wbdata
import pandas as pd

def population(year, sex, age_range, place):
    """
    Return population count for a given year, sex, age_range, and place (WDI via wbdata).

    sex: 'people'/'male'/'female'  (case-insensitive, can also pass 'males','females')
    age_range: must be one of (0,14), (15,64), (65,100), or (0,100)
    place: World Bank code, e.g. 'CUB','USA','WLD','HIC', etc.

    Uses coarse WDI age bins (0-14, 15-64, 65+), so arbitrary ranges are not supported.
    """

    indicators = {
        "SP.POP.TOTL": "total",
        "SP.POP.TOTL.MA.ZS": "male_pct",
        "SP.POP.TOTL.FE.ZS": "female_pct",
        "SP.POP.0014.TO.ZS": "age_0_14_pct",
        "SP.POP.1564.TO.ZS": "age_15_64_pct",
        "SP.POP.65UP.TO.ZS": "age_65_up_pct",
    }

    df = wbdata.get_dataframe(indicators, country=place)
    df.index = pd.to_datetime(df.index).year
    df = df.sort_index()

    if df.empty or year not in df.index:
        raise ValueError(f"No data for place={place}, year={year}")

    row = df.loc[year]
    total = float(row["total"])

    # sex share
    s = str(sex).lower()
    if s in ["people", "all", "total"]:
        sex_share = 1.0
    elif s in ["male", "males", "man", "men"]:
        sex_share = float(row["male_pct"]) / 100.0
    elif s in ["female", "females", "woman", "women"]:
        sex_share = float(row["female_pct"]) / 100.0
    else:
        raise ValueError("sex must be people/male/female")

    # age share
    low, high = age_range
    if (low, high) == (0, 14):
        age_share = float(row["age_0_14_pct"]) / 100.0
    elif (low, high) == (15, 64):
        age_share = float(row["age_15_64_pct"]) / 100.0
    elif (low, high) == (65, 100):
        age_share = float(row["age_65_up_pct"]) / 100.0
    elif (low, high) == (0, 100):
        age_share = 1.0
    else:
        raise ValueError("age_range must be (0,14), (15,64), (65,100), or (0,100)")

    return total * sex_share * age_share

Key '611279067024838417' not in persistent cache.
Key '-3532840903410203748' not in persistent cache.
Key '6752515993659452696' not in persistent cache.
Key '-4031038595067172486' not in persistent cache.
Key '-1220035207549886519' not in persistent cache.
Key '5215110990274055899' not in persistent cache.
Key '-5332548775035304233' not in persistent cache.
Key '1072919073901574592' not in persistent cache.
Key '7678558658024012054' not in persistent cache.
Key '1809573330825704655' not in persistent cache.
Key '-8322353298092923197' not in persistent cache.
Key '8923780036083963948' not in persistent cache.
Key '9087111517950761218' not in persistent cache.
Key '7971753629945722871' not in persistent cache.
Key '4281462373991953045' not in persistent cache.
Key '-3161284732610792981' not in persistent cache.
Key '7124072164458863463' not in persistent cache.
Key '6159356248343464042' not in persistent cache.
Key '-4624198340383161140' not in persistent cache.
Key '-8768318310818203305

## 2A Unit Test

In [3]:
# A2-1: basic sanity: total world population should be in billions
t_wld = population(2000, "people", (0,100), "WLD")
assert t_wld > 5e9, "World population in 2000 should be > 5 billion"

# A2-2: male/female shares should sum close to total
m_wld = population(2000, "male", (0,100), "WLD")
f_wld = population(2000, "female", (0,100), "WLD")
assert abs((m_wld + f_wld) - t_wld) / t_wld < 0.03, "Male+Female should be close to total"

# A2-3: male should be between 45% and 55% of total (very safe band)
assert 0.45 < m_wld / t_wld < 0.55, "Male share should be around 50%"

Key '8939932975823919937' not in persistent cache.


## 3A Population Dataframe

In [5]:
import pandas as pd

def population_dataframe(places, years):
    """
    Return a DataFrame indexed by (place, year).
    Columns are population counts for age-sex groups (3 age bins x 2 sexes).
    """
    rows = []
    for p in places:
        for y in years:
            rows.append({
                "place": p,
                "year": y,
                "Male_0_14": population(y, "male", (0,14), p),
                "Male_15_64": population(y, "male", (15,64), p),
                "Male_65_100": population(y, "male", (65,100), p),
                "Female_0_14": population(y, "female", (0,14), p),
                "Female_15_64": population(y, "female", (15,64), p),
                "Female_65_100": population(y, "female", (65,100), p),
            })
    return pd.DataFrame(rows).set_index(["place", "year"]).sort_index()

In [6]:
df = population_dataframe(
    places=["CUB", "USA"],
    years=range(1960, 2024)
)
df.head()

Male_0_14    Male_15_64    Male_65_100   Female_0_14  \
place year                                                            
CUB   1960  1.332030e+06  2.206805e+06  158503.309123  1.275614e+06   
      1961  1.364712e+06  2.243471e+06  161776.775692  1.309398e+06   
      1962  1.401601e+06  2.277814e+06  165609.216360  1.347305e+06   
      1963  1.442509e+06  2.310876e+06  170201.356426  1.389011e+06   
      1964  1.483939e+06  2.344502e+06  175813.644195  1.431134e+06   

            Female_15_64  Female_65_100  
place year                               
CUB   1960  2.113339e+06  151790.190877  
      1961  2.152540e+06  155219.724308  
      1962  2.189575e+06  159193.783640  
      1963  2.225173e+06  163889.143574  
      1964  2.261074e+06  169557.377758